In [ ]:
import os
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (109 kB/s)
Reading package lists... Done


In [ ]:
#importing yfinance data
import yfinance as yf
celh = yf.download("CELH")
print(celh.head())

[*********************100%%**********************]  1 of 1 completed

                 Open       High        Low      Close  Adj Close  Volume
Date                                                                     
2007-01-22   8.000000  13.333333   8.000000  13.333333  13.333333   15075
2007-01-23  15.000000  15.000000  10.000000  10.600000  10.600000    9330
2007-01-24  10.666667  15.000000  10.666667  11.933333  11.933333   20415
2007-01-25  12.333333  13.400000  12.333333  13.333333  13.333333   12300
2007-01-26  12.800000  12.800000  12.666667  12.733333  12.733333   16275


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("celh").getOrCreate()

In [ ]:
#creating df in spark
pandas_df = celh
pandas_df.reset_index(drop=False,inplace=True)
df = spark.createDataFrame(pandas_df)
df.show(10)

+-----+-------------------+------------------+------------------+------------------+------------------+------------------+------+
|index|               Date|              Open|              High|               Low|             Close|         Adj Close|Volume|
+-----+-------------------+------------------+------------------+------------------+------------------+------------------+------+
|    0|2007-01-22 00:00:00|               8.0|13.333333015441895|               8.0|13.333333015441895|13.333333015441895| 15075|
|    1|2007-01-23 00:00:00|              15.0|              15.0|              10.0|10.600000381469727|10.600000381469727|  9330|
|    2|2007-01-24 00:00:00|10.666666984558105|              15.0|10.666666984558105|11.933333396911621|11.933333396911621| 20415|
|    3|2007-01-25 00:00:00|12.333333015441895|13.399999618530273|12.333333015441895|13.333333015441895|13.333333015441895| 12300|
|    4|2007-01-26 00:00:00|12.800000190734863|12.800000190734863|12.666666984558105|12.733

In [ ]:
#creating temp view
df.createOrReplaceTempView('celh')

In [ ]:
#query to find all dates between Jan 1st, 2020 and March 1st, 2024
all_data = spark.sql("""select * from celh where Date between "2020-01-01" and "2024-03-01" """).show(10)
all_data

+-------------------+------------------+------------------+------------------+------------------+------------------+-------+
|               Date|              Open|              High|               Low|             Close|         Adj Close| Volume|
+-------------------+------------------+------------------+------------------+------------------+------------------+-------+
|2020-01-02 00:00:00|1.6100000143051147|1.6233329772949219|1.5700000524520874|1.5833330154418945|1.5833330154418945| 858300|
|2020-01-03 00:00:00|1.5666669607162476|1.5666669607162476|1.5099999904632568|1.5499999523162842|1.5499999523162842| 876000|
|2020-01-06 00:00:00|1.5266669988632202| 1.553333044052124|1.4966670274734497|1.5066670179367065|1.5066670179367065| 804600|
|2020-01-07 00:00:00|               1.5|               1.5|1.3700000047683716|1.4199999570846558|1.4199999570846558|3549300|
|2020-01-08 00:00:00|1.4233330488204956|1.4833329916000366|1.4199999570846558| 1.463333010673523| 1.463333010673523|1061100|


In [ ]:
#query to find only all closes between Jan 1st, 2020 and March 1st, 2024
closes = spark.sql("""select Close, Date from celh where Date between "2020-01-01" and "2024-03-01" """).show(10)
closes

+------------------+-------------------+
|             Close|               Date|
+------------------+-------------------+
|1.5833330154418945|2020-01-02 00:00:00|
|1.5499999523162842|2020-01-03 00:00:00|
|1.5066670179367065|2020-01-06 00:00:00|
|1.4199999570846558|2020-01-07 00:00:00|
| 1.463333010673523|2020-01-08 00:00:00|
|1.5633330345153809|2020-01-09 00:00:00|
|1.5399999618530273|2020-01-10 00:00:00|
|1.6033329963684082|2020-01-13 00:00:00|
|1.6033329963684082|2020-01-14 00:00:00|
|1.6100000143051147|2020-01-15 00:00:00|
+------------------+-------------------+
only showing top 10 rows



In [ ]:
#query to find only all adj closes between Jan 1st, 2020 and March 1st, 2024
adj_closes = spark.sql("""select `Adj Close`, Date from celh where Date between "2020-01-01" and "2024-03-01" """).show(10)
adj_closes

+------------------+-------------------+
|         Adj Close|               Date|
+------------------+-------------------+
|1.5833330154418945|2020-01-02 00:00:00|
|1.5499999523162842|2020-01-03 00:00:00|
|1.5066670179367065|2020-01-06 00:00:00|
|1.4199999570846558|2020-01-07 00:00:00|
| 1.463333010673523|2020-01-08 00:00:00|
|1.5633330345153809|2020-01-09 00:00:00|
|1.5399999618530273|2020-01-10 00:00:00|
|1.6033329963684082|2020-01-13 00:00:00|
|1.6033329963684082|2020-01-14 00:00:00|
|1.6100000143051147|2020-01-15 00:00:00|
+------------------+-------------------+
only showing top 10 rows



In [ ]:
#query for closes in descending order between the selected dates
biggest_closes = spark.sql("""select Close, Date from celh where Date between "2020-01-01" and "2024-03-01" order by Close desc""").show(10)
biggest_closes

+-----------------+-------------------+
|            Close|               Date|
+-----------------+-------------------+
|81.62000274658203|2024-02-29 00:00:00|
| 79.2699966430664|2024-03-01 00:00:00|
|68.41999816894531|2023-09-07 00:00:00|
| 67.7699966430664|2024-02-28 00:00:00|
| 67.5199966430664|2024-02-27 00:00:00|
| 67.3133316040039|2023-09-14 00:00:00|
|67.20999908447266|2023-09-08 00:00:00|
| 67.0433349609375|2023-09-12 00:00:00|
|66.85333251953125|2023-09-13 00:00:00|
|66.77999877929688|2023-09-06 00:00:00|
+-----------------+-------------------+
only showing top 10 rows

